In [1]:
from PIL import Image
import os
import numpy as np
from feature import NPDFeature
import pickle
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier as Classifier
from ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report

In [2]:
#读取图片、转换为24*24的灰度图、转成NPD特征、写入pickle文件中
face_dirpath = os.path.join('datasets','original','face')
nonface_dirpath = os.path.join('datasets','original','nonface')
face_paths = [os.path.join(face_dirpath,name) for name in os.listdir(face_dirpath)]
nonface_paths = [os.path.join(nonface_dirpath,name) for name in os.listdir(nonface_dirpath)]
examples_path = os.path.join('datasets','feature.pkl')

if not os.path.exists(examples_path):
    face_imgs = [Image.open(path).resize((24,24)).convert('L') for path in face_paths]
    face_examples = [[NPDFeature(np.array(img)).extract(),1] for img in face_imgs]
    nonface_imgs = [Image.open(path).resize((24,24)).convert('L') for path in nonface_paths]
    nonface_examples = [[NPDFeature(np.array(img)).extract(),-1] for img in nonface_imgs]
    examples = face_examples + nonface_examples
    np.random.shuffle(examples)
    with open(examples_path,'wb') as f:
        pickle.dump(examples,f)

In [3]:
#读取数据，划分训练集和验证集
with open(examples_path,'rb') as f:
    examples = pickle.load(f)
features,labels = zip(*examples)
feature_train,feature_valid,label_train,label_valid = train_test_split(np.array(features),np.array(labels),test_size=0.33,random_state=42)

In [7]:
#创建模型、训练、及输出报告
classifier = AdaBoostClassifier(Classifier,8)
classifier.fit(feature_train,label_train)
predict_label = classifier.predict(feature_valid)
report = classification_report(label_valid,predict_label,target_names=['class nonface','class face'])
with open('classifier_report.txt','w') as f:
    f.write(report)

In [6]:
print(report)

               precision    recall  f1-score   support

class nonface       0.95      0.96      0.95       172
   class face       0.96      0.94      0.95       158

     accuracy                           0.95       330
    macro avg       0.95      0.95      0.95       330
 weighted avg       0.95      0.95      0.95       330

